In [18]:
try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')
  

import time
import numpy as np

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

# Holonômico (robotino)

In [24]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')

    robotname = 'robotino'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_oneshot_wait)     
                 
    returnCode, wheel1 = sim.simxGetObjectHandle(clientID, 'wheel0_joint', sim.simx_opmode_oneshot_wait)
    returnCode, wheel2 = sim.simxGetObjectHandle(clientID, 'wheel1_joint', sim.simx_opmode_oneshot_wait)
    returnCode, wheel3 = sim.simxGetObjectHandle(clientID, 'wheel2_joint', sim.simx_opmode_oneshot_wait)
           
    # Robotino
    L = 0.135   # Metros
    r = 0.040   # Metros
               
    # Cinemática Direta
    Mdir = np.array([[-r/np.sqrt(3), 0, r/np.sqrt(3)], [r/3, (-2*r)/3, r/3], [r/(3*L), r/(3*L), r/(3*L)]])
                       
    q = np.array([0, 0, 0])
    
    # Lembrar de habilitar o 'Real-time mode'
    t = 0
    lastTime = time.time()
    while t <= 15:
        
        now = time.time()
        dt = now - lastTime
        
        # Cinemática Direta
        # u = [np.deg2rad(45), np.deg2rad(0), np.deg2rad(-45)]
        
        # Velocidade desejada (x, y, w)    
        if t <= 5:
            qdot = np.array([.2, 0, np.deg2rad(10)])
        elif t <= 10:    
            qdot = np.array([0, -.2, np.deg2rad(-10)])
        else:
            qdot = np.array([-.2, 0, 0])
        
        #qdot = np.array([0, .3, 0])
        
        # Cinemática Inversa
        # w1, w2, w3
        Minv = np.linalg.inv(Rz(q[2]) @ Mdir)
        u = Minv @ qdot

        # Enviando velocidades
        sim.simxSetJointTargetVelocity(clientID, wheel1, u[0], sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(clientID, wheel2, u[1], sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(clientID, wheel3, u[2], sim.simx_opmode_streaming)      
        
        q = q + (Rz(q[2]) @ Mdir @ u)*dt
        
        t = t + dt        
        lastTime = now        
        
    print('==> ', t, q[:2], np.rad2deg(q[2]))
    
    returnCode, pos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)        
    print('Pos: ', pos)

    returnCode, ori = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    print('Ori: ', np.rad2deg(ori))
        
    sim.simxSetJointTargetVelocity(clientID, wheel1, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, wheel2, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, wheel3, 0, sim.simx_opmode_oneshot_wait)
        
    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')

Program started
Connected to remote API server
==>  15.000776290893555 [ 2.88009644e-05 -9.99863434e-01] 0.014843940734821792
Pos:  [0.1772177666425705, -2.0062546730041504, 0.038924433290958405]
Ori:  [-9.93319898e-05  1.48095944e-02 -5.26440352e-01]
Program ended


# Não-holonômico (Pioneer - diferencial)

In [27]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')

    robotname = 'Pioneer_p3dx'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotname, sim.simx_opmode_oneshot_wait)     
        
    returnCode, l_wheel = sim.simxGetObjectHandle(clientID, robotname + '_leftMotor', sim.simx_opmode_oneshot_wait)
    returnCode, r_wheel = sim.simxGetObjectHandle(clientID, robotname + '_rightMotor', sim.simx_opmode_oneshot_wait)    
           
    # Específico do robô
    # https://www.generationrobots.com/media/Pioneer3DX-P3DX-RevA.pdf
    # L = 0.381   # Metros
    # r = 0.0975  # Metros
    
    L = 0.331
    r = 0.09751
    
    # Velocidade desejada (linear, angular)
    v = .3
    w = np.deg2rad(10)

    # Cinemática Inversa
    wr = ((2.0*v) + (w*L))/(2.0*r)
    wl = ((2.0*v) - (w*L))/(2.0*r)    
    u = np.array([wr, wl])

    # Enviando velocidades
    sim.simxSetJointTargetVelocity(clientID, l_wheel, wl, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, r_wheel, wr, sim.simx_opmode_oneshot_wait)
    
    q = np.array([0, 0, 0])
    
    t = 0
    # Lembrar de habilitar o 'Real-time mode'
    startTime=time.time()
    lastTime = startTime
    while t < 10:
        
        now = time.time()
        dt = now - lastTime

        # Cinemática Direta
        Mdir = np.array([[r*np.cos(q[2])/2, r*np.cos(q[2])/2], [r*np.sin(q[2])/2, r*np.sin(q[2])/2], [r/L, -r/L]])
        q = q + (Mdir @ u)*dt
        
        t = t + dt        
        lastTime = now
    
    print('==> ', t, q[:2], np.rad2deg(q[2]))
    
    returnCode, pos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)        
    print('Pos: ', pos)

    returnCode, ori = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    print('Ori: ', np.rad2deg(ori))    
    
    sim.simxSetJointTargetVelocity(clientID, r_wheel, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, l_wheel, 0, sim.simx_opmode_oneshot_wait)
        
    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')

Program started
Connected to remote API server
==>  10.000176191329956 [-4.00007048  0.        ] 0.0
Pos:  [-0.08509187400341034, -1.576438546180725, 0.13864625990390778]
Ori:  [ 0.09061561 -0.07323384 45.73659959]
Program ended


# Para praticar

- Tentar utilizar o modelo do youBot (Kuka) e fazer uma navegação/controle simples
- Tentar utilizar um modelo do tipo Ackermann (e.g., Manta) e fazer uma navegação/controle simples